In [6]:
from keras.layers import *
from keras.models import Model, Sequential, load_model
#from keras import backend as K
#import tensorflow as tf

in_height = 360
in_width =  640

out_height = 720
out_width = 1280

#out_height = in_height
#out_width = in_width

In [2]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"
WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"

In [27]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"
 
def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'
    x = Convolution2D(squeeze, (1, 1), padding='same', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='same', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    return x

def reconstruct_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'
    x = Conv2DTranspose(squeeze, (1, 1), padding='valid', name=s_id + sq1x1,strides=2)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    return x

In [31]:
img_input = Input(shape=(in_height,in_width,3))

x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
x = Activation('relu', name='relu_conv1')(x)
x = MaxPooling2D(pool_size=(3, 3), name='pool1')(x)
 
x = fire_module(x, fire_id=2, squeeze=16, expand=64)
x = fire_module(x, fire_id=3, squeeze=16, expand=64)
x = MaxPooling2D(pool_size=(3, 3), name='pool3')(x)
 
x = fire_module(x, fire_id=4, squeeze=32, expand=128)
x = fire_module(x, fire_id=5, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=(3, 3), name='pool5')(x)
 
x = fire_module(x, fire_id=6, squeeze=48, expand=192)
x = fire_module(x, fire_id=7, squeeze=48, expand=192)
x = fire_module(x, fire_id=8, squeeze=64, expand=256)
x = fire_module(x, fire_id=9, squeeze=64, expand=256)

x = Dropout(0.5, name='drop9')(x)
 
x = Convolution2D(16, (1, 1), padding='same', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
#x = GlobalAveragePooling2D()(x)


x = reconstruct_module(x, fire_id=10, squeeze=16, expand=64)
x = reconstruct_module(x, fire_id=11, squeeze=16, expand=64)
#x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)
 
x = reconstruct_module(x, fire_id=12, squeeze=32, expand=128)
x = reconstruct_module(x, fire_id=13, squeeze=32, expand=128)
#x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

"""
x = reconstruct_module(x, fire_id=14, squeeze=48, expand=192)
x = reconstruct_module(x, fire_id=15, squeeze=48, expand=192)
x = reconstruct_module(x, fire_id=16, squeeze=64, expand=256)
x = reconstruct_module(x, fire_id=17, squeeze=64, expand=256)
"""
x = Dropout(0.5, name='drop10')(x)
 
x = Convolution2D(3, (1, 1), padding='valid', name='conv11')(x)
out = Activation('relu', name='relu_conv11')(x)
 
model = Model(img_input, out, name='squeezenet')
model.summary()

Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 360, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 179, 319, 64) 1792        input_18[0][0]                   
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 179, 319, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 59, 106, 64)  0           relu_conv1[0][0]                 
_________________________________________________________________________________________

In [11]:
import cv2, os
import numpy as np
#import pandas as pd

train_dir = ["../Datasets/DIV2K_train_HR/","C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/animal/",
            "C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/grass/","C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/mountain/",
            "C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/plant/","C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/sky/"]

valid_dir = "../Datasets/DIV2K_valid_HR/"

X = list()
Y = list()
#files = pd.read_csv('image_data.csv').Path
train_files = []
valid_files = os.listdir(valid_dir)

for dirs in train_dir:
    train_files.append((dirs,os.listdir(dirs)))


def getImage(file):
    #‪C:\Users\mayan\Downloads\2-dog.jpg
    frame = cv2.imread(file,1)
    return [np.array([cv2.resize(frame,(in_width,in_height))]),np.array([cv2.resize(frame,(out_width,out_height))])]

def getValidation():
    x_valid,y_valid = [],[]
    
    for file in valid_files:
        frame = cv2.imread(valid_dir + file,1)
        try:
            y_valid.append(cv2.resize(frame,(out_width,out_height)))
            x_valid.append(cv2.resize(frame,(in_width,in_height)))
        except Exception as ex:
            print(ex)
    return np.array(x_valid),np.array(y_valid)


def getData(batch_size=128):
    while True:
        for path,files in train_files:
            for i , file in enumerate(files):
                frame = cv2.imread(path + file,1)
                try:
                    Y.append(cv2.resize(frame,(out_width,out_height)))
                    #Y.append(cv2.resize(frame,(in_width,in_height)))
                    X.append(cv2.resize(frame,(in_width,in_height)))
                    if (i + 1) % batch_size == 0:
                            yield (np.array(X),np.array(Y))
                            X.clear()
                            Y.clear()
                except Exception as ex:
                    print(ex)
x_valid,y_valid = getValidation()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
from keras.optimizers import Adam, Adadelta

datagen = ImageDataGenerator(featurewise_center=False, featurewise_std_normalization=False,horizontal_flip=True,vertical_flip=True)
#for X_, Y_ in datagen.flow(X,Y, batch_size=256):
#autoencoder.fit_generator(getData(8),epochs=10,samples_per_epoch=50)
#autoencoder.compile(optimizer='adam', loss='mean_absolute_error',metrics=['acc'])

for X_, Y_ in getData(256):
    #X_,_ = datagen.flow(X_,X_,batch_size=256)
    hist = autoencoder_model.fit(X_, Y_,
                epochs=10,
                batch_size=16,validation_data=(x_valid,y_valid),shuffle=True)
    clear_output()
    val_accuracy = hist.history['val_acc']
    print(val_accuracy)
    autoencoder_model.save('SqueezeNet-%dx%d-%dx%d.h5'%(in_height,in_width,out_height,out_width))
    autoencoder_model.save_weights('SqueezeNet-%dx%d-%dx%d.h5'%(in_height,in_width,out_height,out_width))
    print("Saved model to disk")

[0.3731248676776886, 0.38969022035598755, 0.4016486406326294, 0.3800550401210785, 0.39714059233665466, 0.40310898423194885, 0.3949790298938751, 0.40048953890800476, 0.41286957263946533, 0.4022974967956543]
Saved model to disk
